In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/RobbieGeoghegan/AB_Testing/master/ab_data.csv")

In [3]:
df.shape

(294478, 5)

In [4]:
df.head(3)

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0


In [5]:
df.group.unique()

array(['control', 'treatment'], dtype=object)

In [6]:
df.landing_page.unique()

array(['old_page', 'new_page'], dtype=object)

In [7]:
df["group"].value_counts()

treatment    147276
control      147202
Name: group, dtype: int64

In [8]:
#some of the control group saw the new_page and some tretment group saw the old_page - delete these instances
mask1 = (df["group"] == "control") & (df["landing_page"] == "new_page")
index_to_drop1 = df[mask1].index
df = df.drop(index_to_drop1)

mask2 = (df["group"] == "treatment") & (df["landing_page"] == "old_page")
index_to_drop2 = df[mask2].index
df = df.drop(index_to_drop2)

print(df.shape)
df["group"].value_counts()

(290585, 5)


treatment    145311
control      145274
Name: group, dtype: int64

In [9]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [10]:
#Check how many duplicated users exist
print(df["user_id"].count())
print(df["user_id"].nunique())

290585
290584


In [11]:
#drop duplicated users
df.drop_duplicates(subset ='user_id',keep ='first',inplace = True)

In [12]:
df.shape

(290584, 5)

In [13]:
df.pivot_table(index='group', values='converted')

,converted
group,
control,0.120386
treatment,0.118808


In [14]:
df.pivot_table(index='group', values='converted' , aggfunc=['sum', 'count'])

,sum,count
,converted,converted
group,,
control,17489,145274
treatment,17264,145310


In [15]:
#Show the % split between users who saw new vs old page
#Calculate pooled probability
mask = (df["group"] == "control")
conversions_control = df["converted"][mask].sum()
total_users_control = df["converted"][mask].count()

In [16]:
mask = (df["group"] == "treatment")
conversions_treatment = df["converted"][mask].sum()
total_users_treatment = df["converted"][mask].count()

In [17]:
print("Split of control users who saw old page vs treatment users who saw new page: ", 
          round(total_users_control / df["converted"].count() * 100, 2), "% ",
          round((total_users_treatment / df["converted"].count()) * 100, 2), "%" ,
     '\n' )

#count number of users who converted in each group
print("Number of control users were put on old page: ", total_users_control)
print("Number of control users who converted on old page: ", conversions_control)
print("Percentage of control users who converted: ", round((conversions_control / total_users_control) * 100, 2), "%")

print('')
print("Number of control users were put on new page: ", total_users_treatment)
print("Number of treatment users who converted on new page: ", conversions_treatment)
print("Percentage of treatment users who converted: ", round((conversions_treatment/ total_users_treatment) * 100, 2), "%")

Split of control users who saw old page vs treatment users who saw new page:  49.99 %  50.01 % 

Number of control users were put on old page:  145274
Number of control users who converted on old page:  17489
Percentage of control users who converted:  12.04 %

Number of control users were put on new page:  145310
Number of treatment users who converted on new page:  17264
Percentage of treatment users who converted:  11.88 %


## Chi Square

In [26]:
'''
contingency = [[A_purchase , A_no_purchase ],
               [B_purchase , B_no_purchase ],
               [C_purchase , C_no_purchase ]]
'''

contingency = [
                [conversions_control, total_users_control-conversions_control],
                [conversions_treatment , total_users_treatment-conversions_treatment]
                ]


from scipy.stats import chi2_contingency

#chi2 -> The test statistic.
#pval -> The p-value of the test
#dof -> Degrees of freedom
#expected -> The expected frequencies, based on the marginal sums of the table.
chi2, pval, dof, expected = chi2_contingency(contingency, correction=False)


print(chi2)
print(pval)
print(dof)
print(expected)

1.7185222540121272
0.18988337448194853
1
[[ 17374.34725243 127899.65274757]
 [ 17378.65274757 127931.34725243]]


In [19]:
S_mean = (conversions_treatment - conversions_control)
S_var = (conversions_control/total_users_control) + (conversions_treatment/total_users_treatment)

z_score = S_mean / np.sqrt(S_var)
print('Z-score: {:0.4f}'.format(z_score))

Z-score: -460.0521


In [20]:
from scipy.stats import norm

print('p-value: {:0.4f}'.format(norm().sf(z_score)))

p-value: 1.0000


## Test parameters

In [21]:
import math
import statsmodels.stats.api as sms
import scipy.stats as st

#Check what sample size is required
baseline_rate = conversions_control / total_users_control
practical_significance = 0.01 #user defined
confidence_level = 0.05 #user defined, for a 95% confidence interval
sensitivity = 0.8 #user defined

effect_size = sms.proportion_effectsize(baseline_rate, baseline_rate + practical_significance)
sample_size = sms.NormalIndPower().solve_power(effect_size = effect_size, power = sensitivity, 
                                               alpha = confidence_level, ratio=1)
print("Required sample size: ", round(sample_size), " per group")

Required sample size:  17209  per group


## A/B test

In [22]:
#calc pooled probability
prob_pooled = (conversions_control + conversions_treatment) / (total_users_control + total_users_treatment)
print(prob_pooled)

0.11959708724499628


In [28]:
#Calculate pooled standard error and margin of error
se_pooled = math.sqrt(prob_pooled * (1 - prob_pooled) * (1 / total_users_control + 1 / total_users_treatment))
z_score = st.norm.ppf(1 - confidence_level / 2)
margin_of_error = se_pooled * z_score

In [33]:
z_score

-460.0521254034697

In [34]:
margin_of_error

0.0023596265703341376

In [26]:
#Calculate dhat, the estimated difference between probability of conversions in the experiment and control groups
d_hat = (conversions_treatment / total_users_treatment) - (conversions_control / total_users_control)

#Test if we can reject the null hypothesis
lower_bound = d_hat - margin_of_error
upper_bound = d_hat + margin_of_error

if practical_significance < lower_bound:
    print("Reject null hypothesis")
else: 
    print("Do not reject the null hypothesis")
    
print("The lower bound of the confidence interval is ", round(lower_bound * 100, 2), "%")
print("The upper bound of the confidence interval is ", round(upper_bound * 100, 2), "%")

Do not reject the null hypothesis
The lower bound of the confidence interval is  -0.39 %
The upper bound of the confidence interval is  0.08 %
